In [ ]:
#calculation reservoir storage using simulation method
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import os
import math
from scipy.optimize import curve_fit


def CS(areas, Asurf, Depth):
    storages = [1./5 * a ** (5./4) * Depth * Asurf ** (-1./4)/1000. for a in areas]
    return storages

def WS_CF(areas, Asurf, Depth):
    storages = [1./4 * a ** (4./3) * Depth * Asurf ** (-1./3)/1000. for a in areas]
    return storages

def BS_CP(areas, Asurf, Depth):
    storages = [2./7 * a ** (7./5) * Depth * Asurf ** (-2./5)/1000. for a in areas]
    return storages

def PS_WF(areas, Asurf, Depth):
    storages = [1./3 * a ** (3./2) * Depth * Asurf ** (-1./2)/1000. for a in areas]
    return storages

def BF_WP(areas, Asurf, Depth):
    storages = [2./5 * a ** (5./3) * Depth * Asurf ** (-2./3)/1000. for a in areas]
    return storages

def PF_BP(areas, Asurf, Depth):
    storages = [1./2 * a ** (2.) * Depth * Asurf ** (-1.)/1000. for a in areas]
    return storages

def PP(areas, Asurf, Depth):
    storages = [2./3 * a ** (3.) * Depth * Asurf ** (-2.)/1000. for a in areas]
    return storages

    
def new_depth(row):
    H, A, V = row[['DAM_HGT_M', 'Area', 'CAP_MCM']]       
    if H == -99 or A == -99 or V == -99:
        return [row['GRAND_ID'], 0, 0, 0]
    
    change = 1
    H = H * 0.95
    if row['CAP_REP'] != -99:
        A = row['Area']
        V = row['CAP_REP']
        H = H * (row['CAP_REP']/row['CAP_MCM']) ** (1/3)
        # change = 1
    else:
        A = row['Area']
        V = row['CAP_MCM']
        H = H 
        # change = 1
    
    if change == 1:
        return [row['GRAND_ID'], H, A, V]
    else:
        return [row['GRAND_ID'], 0, 0, 0]


grand_info = pd.read_csv('/GRanD_reservoirs_v1.3_input_max.csv', sep=',')

newHAV = grand_info.apply(lambda x: new_depth(x), axis = 1)
newHAV = pd.DataFrame(list(newHAV), columns = ['GRAND_ID', 'H', 'A', 'V'])

min_layers = 10
d_layer = 1
thds = [1/5, 1/4, 2/7, 1/3, 2/5, 1/2, 2/3]
mths = ['CS', 'WS_CF', 'BS_CP', 'PS_WF', 'BF_WP', 'PF_BP', 'PP']

newHAV['newH'] = 0
newHAV['method'] = ''

for row in newHAV.itertuples():
    outname = str(int(row.GRAND_ID))
    print('GRAND' + outname)
    [Depth, Asurf, Storage] = [row.H, row.A, row.V]
    if Depth != 0 and Asurf != 0 and Storage!= 0:
        coeff = Storage/Depth/Asurf
        closest = min(range(len(thds)), key=lambda i: abs(thds[i]-coeff))
        
        newHAV.at[row.Index, 'method'] = mths[closest]
        coeff = thds[closest]
        Depth = Storage/Asurf/coeff
        Depth = round(Depth, 6)
        newHAV.at[row.Index, 'newH'] = Depth
        
       
        df = pd.read_csv('/GRSAD_area_time_series/'+ outname +'_intp', header=0, sep='\s+')
        
        if closest == 0:
            storage_ts = CS(df['3water_enh'], Asurf, Depth)
        elif closest == 1:
            storage_ts = WS_CF(df['3water_enh'], Asurf, Depth)
        elif closest == 2:
            storage_ts = BS_CP(df['3water_enh'], Asurf, Depth)
        elif closest == 3:
            storage_ts = PS_WF(df['3water_enh'], Asurf, Depth)
        elif closest == 4:
            storage_ts = BF_WP(df['3water_enh'], Asurf, Depth)
        elif closest == 5:
            storage_ts = PF_BP(df['3water_enh'], Asurf, Depth)
        elif closest == 6:
            storage_ts = PP(df['3water_enh'], Asurf, Depth)
        else:
            areas = []
            
        output = pd.DataFrame([])
        output['Month'] = df['1month']
        output['Area(km2)'] = df['3water_enh']
        output['Storage(km3)'] = storage_ts
        output.to_csv('/Global_Reservoir_Storage_dataset/'+ outname +'.csv')
